In [ ]:
# !pip install gurobipy

In [1]:
from gurobipy import *

## 問題例14-1 問題サイズを大きくする

In [2]:
# nと初期配置を変更しなければいけない
ship_num=10
stack_num=6
height=6
time = 16

O1=[i+1 for i in range(ship_num)]

S=[i+1 for i in range(stack_num)]
H=[i+1 for i in range(height)]
P=[i+1 for i in range(ship_num)]
T=[i+1 for i in range(time)]
n1=[2,2,2,3,3,2,2,3,2,2]
# n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1],n_[O1[3]-1]]

# S,H,P,T,n1,O

In [3]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal

np.random.seed(0)

# 期待値と分散共分散行列の準備
mean = np.arange(1,20,2)
cov=np.zeros((ship_num,ship_num))
for i in range(ship_num):
  for j in range(ship_num):
    if i==j:
      cov[i][i]=4*np.random.randn()
    elif i>j:
      cov[i][j]=np.random.randn()
      cov[j][i]=cov[i][j]

# numpy を用いた生成
size=20
data_1 = np.random.multivariate_normal(mean, cov, size=size)

# print(data_1.shape)

# print(np.mean(data_1, axis=0))

# print(np.cov(data_1, rowvar=False))

# print(data_1)
O_=np.argsort(data_1)
O_=O_+np.ones((size,ship_num)).astype(int)
O=O_.tolist()

# data_2 = multivariate_normal(mean, cov).rvs(size=200)

# print(data_2.shape)

# print(np.mean(data_2, axis=0))

# print(np.cov(data_2, rowvar=False))

<ipython-input-3-43d76238a4bc>:22: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size=size)


In [4]:
a=[]
d={}
for i in O:
  # print(i)
  if not i in a:
    a.append(i)
for j in a:
  d[tuple(j)]=O.count(j)

d


{(1, 2, 3, 4, 5, 6, 8, 7, 9, 10): 1,
 (1, 2, 4, 3, 5, 6, 7, 8, 9, 10): 1,
 (1, 3, 2, 5, 4, 6, 7, 8, 9, 10): 1,
 (1, 2, 3, 4, 5, 7, 10, 6, 9, 8): 1,
 (2, 1, 4, 3, 7, 5, 6, 8, 9, 10): 1,
 (2, 1, 3, 4, 5, 7, 9, 6, 8, 10): 1,
 (2, 1, 3, 4, 5, 7, 6, 9, 8, 10): 2,
 (1, 2, 5, 4, 3, 6, 7, 8, 9, 10): 1,
 (1, 2, 3, 5, 4, 6, 8, 7, 10, 9): 1,
 (1, 2, 4, 3, 5, 6, 7, 8, 10, 9): 1,
 (3, 1, 2, 4, 5, 6, 7, 10, 9, 8): 1,
 (1, 2, 3, 4, 5, 8, 7, 6, 9, 10): 1,
 (1, 2, 3, 6, 5, 4, 7, 9, 8, 10): 1,
 (2, 1, 3, 5, 4, 6, 7, 9, 8, 10): 1,
 (1, 3, 2, 4, 6, 5, 7, 8, 9, 10): 1,
 (2, 1, 3, 5, 6, 4, 8, 7, 9, 10): 1,
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10): 1,
 (3, 2, 1, 5, 4, 7, 6, 9, 8, 10): 1,
 (1, 6, 2, 4, 3, 5, 8, 7, 10, 9): 1}

In [5]:
m=Model("IP3")

alpha=m.addVar(vtype="C")

# 変数の定義
x,y,z,w={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      z[s,h,i]=m.addVar(vtype="c",lb=0)

for i in range(len(O)):
  w[i]=m.addVar(vtype="C",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      for j,p in enumerate(o):
        m.addConstr(z[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

for i in range(len(O)):
  m.addConstr(w[i]>=quicksum(z[s,h,i] for s in S for h in H if h!=1)-alpha)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-30


In [6]:
# 初期配置の制約

m.addConstr(x[1,1,O1[9],1]==1)
m.addConstr(x[1,2,O1[7],1]==1)
m.addConstr(x[1,3,O1[5],1]==1)
m.addConstr(x[1,4,O1[3],1]==1)
m.addConstr(x[2,1,O1[8],1]==1)
m.addConstr(x[2,2,O1[8],1]==1)
m.addConstr(x[2,3,O1[3],1]==1)
m.addConstr(x[2,4,O1[1],1]==1)
m.addConstr(x[3,1,O1[5],1]==1)
m.addConstr(x[3,2,O1[9],1]==1)
m.addConstr(x[3,3,O1[4],1]==1)
m.addConstr(x[3,4,O1[2],1]==1)
m.addConstr(x[3,5,O1[0],1]==1)
m.addConstr(x[4,1,O1[7],1]==1)
m.addConstr(x[4,2,O1[4],1]==1)
m.addConstr(x[4,3,O1[0],1]==1)
m.addConstr(x[4,4,O1[2],1]==1)
m.addConstr(x[5,1,O1[6],1]==1)
m.addConstr(x[5,2,O1[1],1]==1)
m.addConstr(x[6,1,O1[7],1]==1)
m.addConstr(x[6,2,O1[4],1]==1)
m.addConstr(x[6,3,O1[6],1]==1)
m.addConstr(x[6,4,O1[3],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [7]:
beta=0.8
m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(w[i] for i in range(len(O)))/((1-beta)*len(O)))

In [8]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22156 rows, 6831 columns and 220053 nonzeros
Model fingerprint: 0xe9dd03bc
Variable types: 621 continuous, 6210 integer (6210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 2399 rows and 941 columns
Presolve time: 1.16s
Presolved: 19757 rows, 5890 columns, 199255 nonzeros
Variable types: 621 continuous, 5269 integer (5269 binary)
Found heuristic solution: objective 4.0000000

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    3849    3.6378636e-04   2.531866e+04   0.000000e+00      5s
    6049    4.0285721e-04   1.829075e+04   0.000000e+00     10s
    7649    4.1843909e-04   6.047494e+04   0.000000e+00     15s
    9129    4.2780806e-04   3.513364e+03   0.000000e+

In [34]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    if t==1:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t>=2:
      if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in z:
    if z[s,h,i].X>EPS:
      print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

x[ 1, 1, 4, 1]=1.0
x[ 1, 2, 4, 1]=1.0
x[ 1, 3, 1, 3]=1.0
x[ 1, 3, 3, 1]=1.0
x[ 2, 1, 4, 1]=1.0
x[ 2, 2, 3, 1]=1.0
x[ 2, 3, 2, 1]=1.0
x[ 3, 1, 3, 1]=1.0
x[ 3, 2, 3, 1]=1.0
x[ 3, 3, 3, 2]=1.0
x[ 3, 4, 3, 5]=1.0
x[ 4, 1, 2, 1]=1.0
x[ 4, 2, 1, 1]=1.0
x[ 4, 2, 2, 4]=1.0
y[ 1, 3, 1]=1.0
y[ 2, 3, 4]=1.0
y[ 2, 4, 3]=1.0
y[ 4, 1, 2]=1.0
the objective function 4e-05
VaR= 0.0


In [ ]:
O